In [24]:
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
import pandas as pd
from time import sleep
from tqdm import tqdm


creds = None
with open('../../secret.json') as creds_file:
    creds = json.load(creds_file)


# Set up the Spotify client
client_credentials_manager = SpotifyClientCredentials(client_id=creds['client_id'], client_secret=creds['client_secret'])
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

print('hi')
print(sp)


hi


In [25]:
# import time
# 
# artist_name = "Linkin Park"
# song_name = "Breaking the Habit"
# 
# try:
#     result = sp.search(q=f'track:{song_name} artist:{artist_name}', type='track')
#     print(result)
# except spotipy.SpotifyException as e:
#     print(f"An error occurred: {e}")
# finally:
#     # Introduce a delay between requests (e.g., 1 second)
#     time.sleep(1)


In [26]:
def get_song_release_year(song_name, artist_name):
    sleep(.5)
    # Search for the song
    # trying this search for faster queries hopefully
    # search(q, limit=10, offset=0, type='track', market=None)
    # let q = String.init(format:"artist:%@ track:%@",artistName,trackName)
    # https://stackoverflow.com/questions/42766000/spotipy-how-do-i-search-by-both-artist-and-song
    q = "artist:{} track:{}".format(artist_name, song_name)
    results = sp.search(q, limit=1, offset=0, type='track', market=None)

    # Extract the release year from the first search result
    if len(results['tracks']['items']) > 0:
        release_date = results['tracks']['items'][0]['album']['release_date']
        release_year = release_date.split('-')[0]
        return release_year
    else:
        # Return None if no results were found
        return None

In [6]:
artist_name = "Linkin Park"
song_name = "Breaking the Habit"
try:
    print(get_song_release_year(song_name, artist_name))
except Exception as e:
    print(str(e))



In [27]:
cddb_df = pd.read_csv('../../cleaning_stages/3_category_genre/cddb_2.tsv', sep='\t')
tracks_df = pd.read_csv('../../cleaning_stages/3_category_genre/cddb_tracks.tsv', sep='\t')

# not updating tracks here so can save to next cleaning stage
tracks_df.to_csv('../../cleaning_stages/4_year_repair/cddb_tracks.tsv', sep="\t", index=False)

cddb_df_trim = cddb_df[cddb_df['year'].isnull()]
cddb_df_trim = cddb_df_trim[['artist', 'tracks', 'year', 'id']]

In [28]:
# lists are much faster than dataframes, converting
cddb_list = cddb_df_trim.values.tolist()
tracks_list = tracks_df.values.tolist()

print(f'num null years: ${len(cddb_list)}')

num null years: $4532


In [29]:
# list should have 
n = 100
   
# using list comprehension 
cddb_list_chunks = [cddb_list[i:i + n] for i in range(0, len(cddb_list), n)] 
print(len(cddb_list_chunks))

46


In [33]:
def start_repair(cd_list, tracks_list):
    num_repairs = 0
    num_not_repairable = 0
    
    total_rows = len(cd_list)
    for cddb_row in tqdm(cd_list, total=total_rows, desc='Processing rows', ncols=100):
    #for cddb_row in cd_list:
  
        artist_name = cddb_row[0]
        album_id = cddb_row[1]

        # Grab the tracks for this album
        tracks_for_album = []
        for row in tracks_list:
            if row[1] == album_id:
                tracks_for_album.append(row[2])
            # stop iterating if album id is greater
            elif row[1] > int(album_id):
                break
                
        # tracks_for_album = tracks_df[tracks_df['Album Id'] == cddb_row['tracks']]
        
        if len(tracks_for_album) == 0:
            # No song for track something is wrong
            # print('TOSS: ' + str(cddb_row['id']) + ' ---- ' + artist_name)
            num_not_repairable += 1
            cd_list.remove(cddb_row)
            continue
        elif len(tracks_for_album) >= 5:
            # if first 5 songs fail then odds are the whole row fill fail
            tracks_for_album = tracks_for_album[:5]
        
        # old code
        #for _, track_row in enumerate(tracks_for_album):
        #    song_name = track_row['Name']
        #    fix = get_song_release_year(song_name, artist_name)
        #    if fix:
        #        # print(f'{artist_name} ---- {song_name} ---- {fix}')
        #        cddb_row['year'] = fix
        #        num_repairs += 1
        #        break # we found one we are clean
        
        for track in tracks_for_album:
            fix = get_song_release_year(track, artist_name)
            if fix:
                # print(f'{artist_name} ---- {song_name} ---- {fix}')
                # cddb_row[2] = fix
                num_repairs += 1
                # update cddb_df directly
                idx_list = cddb_df.index[cddb_df['id'] == cddb_row[3]].tolist()
                idx = idx_list[0]
                cddb_df.at[idx,'year'] = fix
                break # we found one we are clean
        
    print(f'Done! num repairs: {num_repairs}, num not repairable: {num_not_repairable}')
    

In [32]:
# attemp to write output every 100 fixes 
itr = 0
for chunk in cddb_list_chunks:
    start_repair(chunk, tracks_list)
    print("finished chunk: " + str(itr))
    cddb_df.to_csv('../../cleaning_stages/4_year_repair/cddb_3.tsv', sep="\t", index=False)
    itr = itr + 1
   

Processing rows: 100%|████████████████████████████████████████████| 100/100 [03:30<00:00,  2.10s/it]


Done! num repairs: 40, num not repairable: 0
finished chunk: 0


Processing rows: 100%|████████████████████████████████████████████| 100/100 [03:10<00:00,  1.91s/it]


Done! num repairs: 50, num not repairable: 0
finished chunk: 1


Processing rows: 100%|████████████████████████████████████████████| 100/100 [03:27<00:00,  2.07s/it]


...

Done! num repairs: 47, num not repairable: 0
finished chunk: 43


Processing rows: 100%|████████████████████████████████████████████| 100/100 [01:48<00:00,  1.09s/it]


Done! num repairs: 84, num not repairable: 0
finished chunk: 44


Processing rows: 100%|██████████████████████████████████████████████| 32/32 [00:39<00:00,  1.24s/it]

Done! num repairs: 25, num not repairable: 0
finished chunk: 45


In [ ]:
# cddb_df[['artist', 'tracks', 'year']]
# cddb_df[[0, 1, 2]]

# tracks_df[['id', 'album id', 'name']]
# tracks_df[[0, 1, 2]]

# data.at[6,'NAME']='Safa'
# df.index[df['BoolCol']].tolist()